# Coursera Capstone Project
## The Battle of Neighborhoods
### Upload Required Libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans
import lxml.html as lh
import bs4 as bs
import urllib.request

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
from IPython.display import HTML
import base64

# Extra Helper scripts to generate download links for saved dataframes in csv format.
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

### Downloading Singapore towns list with and median residential rental prices

In [3]:
import zipfile
import os
!wget -q -O 'median-rent-by-town-and-flat-type.zip' "https://data.gov.sg/dataset/b35046dc-7428-4cff-968d-ef4c3e9e6c99/download"
zf = zipfile.ZipFile('./median-rent-by-town-and-flat-type.zip')
sgp_median_rent_by_town_data = pd.read_csv(zf.open("median-rent-by-town-and-flat-type.csv"))
sgp_median_rent_by_town_data.rename(columns = {'town':'Town'}, inplace = True)
sgp_median_rent_by_town_data.head()

,quarter,Town,flat_type,median_rent
0,2005-Q2,ANG MO KIO,1-RM,na
1,2005-Q2,ANG MO KIO,2-RM,na
2,2005-Q2,ANG MO KIO,3-RM,800
3,2005-Q2,ANG MO KIO,4-RM,950
4,2005-Q2,ANG MO KIO,5-RM,-


### Data Cleanup and re-grouping
#### The retrieved table contains some un-wanted entries and needs some cleanup

In [4]:
# Drop rows with rental price == 'na'.
sgp_median_rent_by_town_data_filter=sgp_median_rent_by_town_data[~sgp_median_rent_by_town_data['median_rent'].isin(['-','na'])]

# Take the most recent report which is "2018-Q2"
sgp_median_rent_by_town_data_filter=sgp_median_rent_by_town_data_filter[sgp_median_rent_by_town_data_filter['quarter'] == "2019-Q4"]

# Now that all rows reports are "2018-Q2", we dont need this column anymore.
sgp_median_rent_by_town_data_filter=sgp_median_rent_by_town_data_filter.drop(['quarter'], axis=1)

# Ensure that median_rent column is float64.
sgp_median_rent_by_town_data_filter['median_rent']=sgp_median_rent_by_town_data_filter['median_rent'].astype(np.float64)

In [5]:
singapore_average_rental_prices_by_town = sgp_median_rent_by_town_data_filter.groupby(['Town'])['median_rent'].mean().reset_index()
singapore_average_rental_prices_by_town

,Town,median_rent
0,ANG MO KIO,2033.333333
1,BEDOK,2112.500000
2,BISHAN,2200.000000
3,BUKIT BATOK,1950.000000
4,BUKIT MERAH,2175.000000
5,BUKIT PANJANG,1757.500000
6,CENTRAL,2375.000000
7,CHOA CHU KANG,1883.333333
8,CLEMENTI,2250.000000
9,GEYLANG,2183.333333


### Retrieve town coordinates

In [6]:
    geo = Nominatim(user_agent="Mypythonapi")
    for idx,town in singapore_average_rental_prices_by_town['Town'].iteritems():
        coord = geo.geocode(town + ' ' + "Singapore", timeout = 10)
        if coord:
            singapore_average_rental_prices_by_town.loc[idx,'Latitude'] = coord.latitude
            singapore_average_rental_prices_by_town.loc[idx,'Longitude'] = coord.longitude
        else:
            singapore_average_rental_prices_by_town.loc[idx,'Latitude'] = NULL
            singapore_average_rental_prices_by_town.loc[idx,'Longitude'] = NULL

In [7]:
singapore_average_rental_prices_by_town.set_index("Town")

,median_rent,Latitude,Longitude
Town,,,
ANG MO KIO,2033.333333,1.370073,103.849516
BEDOK,2112.500000,1.323976,103.930216
BISHAN,2200.000000,1.351452,103.848250
BUKIT BATOK,1950.000000,1.349057,103.749591
BUKIT MERAH,2175.000000,1.280628,103.830591
BUKIT PANJANG,1757.500000,1.378629,103.762136
CENTRAL,2375.000000,1.340863,103.830392
CHOA CHU KANG,1883.333333,1.384749,103.744534
CLEMENTI,2250.000000,1.315100,103.765231


### Generate Singapore basemap

In [8]:
geo = Nominatim(user_agent='My-IBMNotebook')
address = 'Singapore'
location = geo.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore {}, {}.'.format(latitude, longitude))

# create map of Singapore using latitude and longitude values
map_singapore = folium.Map(location=[latitude, longitude],tiles="OpenStreetMap", zoom_start=12)

# add markers to map
for lat, lng, town in zip(
    singapore_average_rental_prices_by_town['Latitude'],
    singapore_average_rental_prices_by_town['Longitude'],
    singapore_average_rental_prices_by_town['Town']):
    label = town
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_singapore)
map_singapore

The geograpical coordinate of Singapore 1.357107, 103.8194992.


In [9]:
fileName = "singapore_average_rpbt.csv"
linkName = "Singapore Average Rental Prices"
create_download_link(singapore_average_rental_prices_by_town,linkName,fileName)

## Segmenting and Clustering Towns in Singapore
### Exploring Town in Singapore using FourSquare
#### Foursquare Credentials and Version

In [10]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: Q0DLPFSUOVIJZH4JDQ2JTRXT4ZUAXFSARHAQDGZ4C1VFOA2I
CLIENT_SECRET:I4MNQ1BZEZOQNT21T11GRIINKBYEBICZAYFXHP0XJ1BFNXKV


### Defining function to get the top 100 venues within a radius of 500 meters and clean the json and structure it into a pandas dataframe

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
SG_venues = getNearbyVenues(names=singapore_average_rental_prices_by_town['Town'],
                                   latitudes=singapore_average_rental_prices_by_town['Latitude'],
                                   longitudes=singapore_average_rental_prices_by_town['Longitude']
                                  )

ANG MO KIO
BEDOK
BISHAN
BUKIT BATOK
BUKIT MERAH
BUKIT PANJANG
CENTRAL
CHOA CHU KANG
CLEMENTI
GEYLANG
HOUGANG
JURONG EAST
JURONG WEST
KALLANG/WHAMPOA
MARINE PARADE
PASIR RIS
PUNGGOL
QUEENSTOWN
SEMBAWANG
SENGKANG
SERANGOON
TAMPINES
TOA PAYOH
WOODLANDS
YISHUN


In [13]:
print(SG_venues.shape)
SG_venues.head()

(936, 7)


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ANG MO KIO,1.370073,103.849516,FairPrice Xtra,1.369279,103.848886,Supermarket
1,ANG MO KIO,1.370073,103.849516,Old Chang Kee,1.369094,103.848389,Snack Place
2,ANG MO KIO,1.370073,103.849516,Subway,1.369136,103.847612,Sandwich Place
3,ANG MO KIO,1.370073,103.849516,MOS Burger,1.369170,103.847831,Burger Joint
4,ANG MO KIO,1.370073,103.849516,NTUC FairPrice,1.371507,103.847082,Supermarket


#### Check how many venues were returned for each Town

In [14]:
SG_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
ANG MO KIO,42,42,42,42,42,42
BEDOK,58,58,58,58,58,58
BISHAN,44,44,44,44,44,44
BUKIT BATOK,22,22,22,22,22,22
BUKIT MERAH,39,39,39,39,39,39
BUKIT PANJANG,43,43,43,43,43,43
CENTRAL,4,4,4,4,4,4
CHOA CHU KANG,19,19,19,19,19,19
CLEMENTI,64,64,64,64,64,64


In [15]:
print('There are {} uniques categories.'.format(len(SG_venues['Venue Category'].unique())))

There are 142 uniques categories.


### Analyze Each Town

In [16]:
# one hot encoding
SG_onehot = pd.get_dummies(SG_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SG_onehot['Town'] = SG_venues['Town'] 

# move neighborhood column to the first column
fixed_columns = [SG_onehot.columns[-1]] + list(SG_onehot.columns[:-1])
SG_onehot = SG_onehot[fixed_columns]

SG_onehot.head()

,Town,Accessories Store,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Basketball Court,Bay,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,German Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Health & Beauty Service,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lottery Retailer,Lounge,Malay Restaurant,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Neighborhood,Night Market,Noodle House,Outlet Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Ramen Restaurant,Recreation Center,Reservoir,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ANG MO KIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
SG_onehot.shape

(936, 143)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
SG_grouped = SG_onehot.groupby('Town').mean().reset_index()
SG_grouped

,Town,Accessories Store,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Basketball Court,Bay,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,German Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hainan Restaurant,Halal Restaurant,Health & Beauty Service,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Light Rail Station,Lottery Retailer,Lounge,Malay Restaurant,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Neighborhood,Night Market,Noodle House,Outlet Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Ramen Restaurant,Recreation Center,Reservoir,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,ANG MO KIO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.02381,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.047619,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.095238,0.023810,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.023810,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.023810,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.02381,0.000000,0.02381,0.000000,0.023810,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.023810,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.00,0.000000,0.000000,0.000000,0.023810,0.000,0.000000,0.023810,0.000000,0.000000,0.023810,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.047619,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,BEDOK,0.000000,0.017241,0.000000,0.000000,0.051724,0.000000,0.017241,0.00000,0.000000,0.00,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.017241,0.000000,0.017241,0.017241,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.068966,0.017241,0.068966,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.017241,0.051724,0.000000,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.017241,0.017241,0.017241,0.017241,0.017241

In [19]:
SG_grouped.shape

(25, 143)

#### Print each Town along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in SG_grouped['Town']:
    print("----"+hood+"----")
    temp = SG_grouped[SG_grouped['Town'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ANG MO KIO----
             venue  freq
0       Food Court  0.10
1      Coffee Shop  0.10
2      Supermarket  0.05
3  Bubble Tea Shop  0.05
4      Snack Place  0.05


----BEDOK----
                 venue  freq
0   Chinese Restaurant  0.07
1          Coffee Shop  0.07
2           Food Court  0.05
3  Japanese Restaurant  0.05
4     Asian Restaurant  0.05


----BISHAN----
                venue  freq
0          Food Court  0.11
1  Chinese Restaurant  0.07
2         Coffee Shop  0.07
3     Bubble Tea Shop  0.07
4                Café  0.07


----BUKIT BATOK----
                  venue  freq
0           Coffee Shop  0.18
1  Fast Food Restaurant  0.09
2            Food Court  0.09
3    Chinese Restaurant  0.09
4         Bowling Alley  0.05


----BUKIT MERAH----
                venue  freq
0    Asian Restaurant  0.10
1  Chinese Restaurant  0.10
2                Café  0.10
3          Food Court  0.08
4           Bookstore  0.05


----BUKIT PANJANG----
                  venue  freq
0      Sus

#### function to sort the venues in descending order

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creat new dataframe and display the top 10 venues for each Town

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Town_venues_sorted = pd.DataFrame(columns=columns)
Town_venues_sorted['Town'] = SG_grouped['Town']

for ind in np.arange(SG_grouped.shape[0]):
    Town_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SG_grouped.iloc[ind, :], num_top_venues)

Town_venues_sorted.head()

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,Coffee Shop,Food Court,Supermarket,Snack Place,Japanese Restaurant,Bubble Tea Shop,Dessert Shop,Modern European Restaurant,Chinese Restaurant,Ramen Restaurant
1,BEDOK,Chinese Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Sandwich Place,Food Court,Café,Supermarket,Fast Food Restaurant,Malay Restaurant
2,BISHAN,Food Court,Bubble Tea Shop,Café,Coffee Shop,Chinese Restaurant,Cosmetics Shop,Supermarket,Ice Cream Shop,Japanese Restaurant,Shopping Mall
3,BUKIT BATOK,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Food Court,Department Store,Grocery Store,Sandwich Place,Multiplex,Café,Shopping Mall
4,BUKIT MERAH,Café,Asian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Bookstore,Noodle House,Restaurant,Yoga Studio,Bus Station


## Cluster Towns
### Run k-means to cluster the Towns into 5 clusters

In [23]:
# set number of clusters
kclusters = 5

SG_grouped_clustering = SG_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SG_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 0], dtype=int32)

#### create a new dataframe that includes the cluster as well as the top 10 venues for each Town

In [24]:
# add clustering labels
Town_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SG_merged = singapore_average_rental_prices_by_town

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
SG_merged = singapore_average_rental_prices_by_town.join(Town_venues_sorted.set_index('Town'), on='Town')

SG_merged.head() # check the last columns!

,Town,median_rent,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,2033.333333,1.370073,103.849516,1,Coffee Shop,Food Court,Supermarket,Snack Place,Japanese Restaurant,Bubble Tea Shop,Dessert Shop,Modern European Restaurant,Chinese Restaurant,Ramen Restaurant
1,BEDOK,2112.500000,1.323976,103.930216,1,Chinese Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Sandwich Place,Food Court,Café,Supermarket,Fast Food Restaurant,Malay Restaurant
2,BISHAN,2200.000000,1.351452,103.848250,1,Food Court,Bubble Tea Shop,Café,Coffee Shop,Chinese Restaurant,Cosmetics Shop,Supermarket,Ice Cream Shop,Japanese Restaurant,Shopping Mall
3,BUKIT BATOK,1950.000000,1.349057,103.749591,1,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Food Court,Department Store,Grocery Store,Sandwich Place,Multiplex,Café,Shopping Mall
4,BUKIT MERAH,2175.000000,1.280628,103.830591,1,Café,Asian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Bookstore,Noodle House,Restaurant,Yoga Studio,Bus Station


### Visualize the resulting clusters

In [25]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SG_merged['Latitude'], SG_merged['Longitude'], SG_merged['Town'], SG_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
### Cluster 1

In [26]:
SG_merged.loc[SG_merged['Cluster Labels'] == 0, SG_merged.columns[[0,1] + list(range(5, SG_merged.shape[1]))]]

,Town,median_rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,GEYLANG,2183.333333,Chinese Restaurant,Noodle House,Asian Restaurant,Food Court,Vegetarian / Vegan Restaurant,Seafood Restaurant,Grocery Store,Dim Sum Restaurant,Pool,Cantonese Restaurant
17,QUEENSTOWN,2170.000000,Food Court,Chinese Restaurant,Noodle House,Steakhouse,Pool,Train Station,Italian Restaurant,Spa,Seafood Restaurant,Stadium


### Cluster 2

In [27]:
SG_merged.loc[SG_merged['Cluster Labels'] == 1, SG_merged.columns[[0,1] + list(range(5, SG_merged.shape[1]))]]

,Town,median_rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANG MO KIO,2033.333333,Coffee Shop,Food Court,Supermarket,Snack Place,Japanese Restaurant,Bubble Tea Shop,Dessert Shop,Modern European Restaurant,Chinese Restaurant,Ramen Restaurant
1,BEDOK,2112.500000,Chinese Restaurant,Coffee Shop,Asian Restaurant,Japanese Restaurant,Sandwich Place,Food Court,Café,Supermarket,Fast Food Restaurant,Malay Restaurant
2,BISHAN,2200.000000,Food Court,Bubble Tea Shop,Café,Coffee Shop,Chinese Restaurant,Cosmetics Shop,Supermarket,Ice Cream Shop,Japanese Restaurant,Shopping Mall
3,BUKIT BATOK,1950.000000,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Food Court,Department Store,Grocery Store,Sandwich Place,Multiplex,Café,Shopping Mall
4,BUKIT MERAH,2175.000000,Café,Asian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Bookstore,Noodle House,Restaurant,Yoga Studio,Bus Station
5,BUKIT PANJANG,1757.500000,Coffee Shop,Fast Food Restaurant,Café,Sushi Restaurant,Asian Restaurant,Ramen Restaurant,Noodle House,Pharmacy,Shopping Mall,Fried Chicken Joint
7,CHOA CHU KANG,1883.333333,Coffee Shop,Fast Food Restaurant,Supermarket,Café,Food Court,Smoke Shop,Furniture / Home Store,Bubble Tea Shop,Sandwich Place,Bus Station
8,CLEMENTI,2250.000000,Coffee Shop,Food Court,Noodle House,Dessert Shop,Asian Restaurant,Chinese Restaurant,Gym,Dim Sum Restaurant,Electronics Store,Fast Food Restaurant
10,HOUGANG,1982.500000,Food Court,Chinese Restaurant,Fast Food Restaurant,Supermarket,Shopping Mall,Coffee Shop,Café,Dessert Shop,Pharmacy,Bookstore
11,JURONG EAST,2100.000000,Japanese Restaurant,Chinese Restaurant,Coffee Shop,Café,Food Court,Shopping Mall,Sushi Restaurant,Department Store,Korean Restaurant,Multiplex


### Cluster 3

In [29]:
SG_merged.loc[SG_merged['Cluster Labels'] == 2, SG_merged.columns[[0,1] + list(range(5, SG_merged.shape[1]))]]

,Town,median_rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,KALLANG/WHAMPOA,2293.333333,Chinese Restaurant,Coffee Shop,Pet Store,Food Court,Park,Convenience Store,Restaurant,Bus Station,Asian Restaurant,Photography Studio


### Cluster 4

In [30]:
SG_merged.loc[SG_merged['Cluster Labels'] == 3, SG_merged.columns[[0,1] + list(range(5, SG_merged.shape[1]))]]

,Town,median_rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,SEMBAWANG,1900.0,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Snack Place,Bus Station,Bus Stop,Bistro,Supermarket,Shopping Mall,BBQ Joint
21,TAMPINES,2100.0,Coffee Shop,Supermarket,Chinese Restaurant,Fast Food Restaurant,Seafood Restaurant,Indian Restaurant,Bus Station,Food Court,Noodle House,Food Truck


### Cluster 5

In [31]:
SG_merged.loc[SG_merged['Cluster Labels'] == 4, SG_merged.columns[[0,1] + list(range(5, SG_merged.shape[1]))]]

,Town,median_rent,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,CENTRAL,2375.0,Bay,Reservoir,Trail,Park,Yoga Studio,Food Truck,Food Court,Flower Shop,Fish & Chips Shop,Filipino Restaurant


## Thank you!